## Scrapy

Unlike the other Python packages we have used so far, scrapy comes with its own API. While we haven't covered API's just yet, what that means practically is that scrapy makes it easy to write and debug spiders (the little programs that crawl/scrape the website).  You can also run scrapy in the command line or jupyter notebook.  For ease of illustration, the examples we use will be in the jupyter notebook.  If you want to pursue scraping further, we enourage you to use the [API](https://doc.scrapy.org/en/latest/topics/api.html). You can also see an example of a project that uses that API [here](https://doc.scrapy.org/en/latest/intro/examples.html)

Here we will write a scraper to pull data from the [_Everyday Sexism Project_](https://everydaysexism.com/), which has been collecting stories about times people have experienced or observed sexism since 2012. User-provided entries are stored and displayed as simple text entries, with user-provided categories. There is no API or other apparent method to access the data aside from the webpage, and experiences of sexism are so subjective that it is difficult to imagine getting similar-quality information through a questionnaire or other easy-to-analyse data collection method. Yet as presented, it isn’t possible to see any overarching trends beyond the sheer quantity of entries.

A data scientist with a specific interest in learning more about experiences of sexism might want to write a scraper for the _Everyday Sexism Project_. A good scraper will pull data from every page of the ESP website. We want to save the entry text, any category labels, date posted, and the name provided by the poster. Our scraper will scrape each page, then find the URL for the previous page and call itself again for that page.


## Basic Scraper

Let's start with a simple scraper that just saves the entire webpage as an HTML file.  Note that Scrapy is built on top of the [Twisted](http://twistedmatrix.com/trac/) [reactor](http://stackoverflow.com/questions/35111265/how-does-pythons-twisted-reactor-work) which is event-driven and cannot be restarted within a session. What this means for us is that after each time you run a Scrapy script example here, you'll need to restart the kernel before you run another Scrapy script. Just another reason to use scrapy in the command line...

In a Scrapy-made scraper, you provide one or more starting URLs. The scraper then sends a request to the server for the information at that URL. The server provides a response, which is either the information requested or an error code. Scrapy then processes the response in accordance with the instructions you've given it.

In [1]:
# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = [
        'http://www.everydaysexism.com',
    ]

    # What to do with the URL.  Here, we tell it to download all the code and save
    # it to the mainpage.html file
    def parse(self, response):
        with open('mainpage.html', 'wb') as f:
            f.write(response.body)


# Instantiate our crawler.
process = CrawlerProcess()

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()


2017-08-25 11:44:35 [scrapy.utils.log] INFO: Scrapy 1.3.2 started (bot: scrapybot)
2017-08-25 11:44:35 [scrapy.utils.log] INFO: Overridden settings: {}
2017-08-25 11:44:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2017-08-25 11:44:35 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddle

If that worked, you should now have a file called 'mainpage.html' saved to your machine that contains all the code from www.everydaysexism.com.  Talk to your mentor if you are having problems with this step, and make sure it is working before you move forward.

By default Scrapy gives detailed logging, which you should see above in red.

Oh, and remember, you'll need to restart your kernel if you want to rerun a Scrapy script. If you get an error, the first thing to try is restarting your kernel.

## Parsing HTML

Now that we know the spider can pull the code, we need to _do_ something with it.

Scraping the web means learning a bit about web technologies: HTML, XML, and CSS. This code contains the content and display information for every web page. You'll need to get familiar with the structure of your document and the code that identifies the piece(s) of the web page you want. To see the code for an entry on www.everydaysexism.com, open the page in the web browser of your choice and hover over the brownish background rectangle that defines an entry.  Left-click, then select "Inspect Element" or some variant- [the terminology will vary depending on your browser](https://victorfont.com/how-to-use-your-browsers-inspect-tool/). In Chrome, the command is "Inspect". This will display your browser's developer tools alongside or below the rendered web page. The "Elements" tab of your developer tools will show the code for the page. The particular element you left-clicked on will be highlighted. As you move around in the code, different parts of the webpage will be highlighted, letting you know what that piece of code refers to.

![Inspecting web page elements](assets/developer_tools.gif)

If you've done it right, your click should have highlighted code starting with `<article id="post-`.  Each submission on the page is contained within an `<article>` element. Nested within each article is a `<header>` element, a `<section>` element (with the class `"entry-content"`), and a `<footer>` element.  Look and click around a bit and see if you can find the information we want to extract from each submission: (1) The author, (2) the date it was posted, (3) the text content, and (4) any tags associated with it. We're lucky that every submission has a standardized format, so we can use the code for this one submission as a template to build our scraper's parsing instructions.

So how do we get the information out of the HTML code and into our parser?  Often, people just learning to scrape think "a ha! I will use regular expressions to pick out what I want!"  This [is not a good idea](https://blog.codinghorror.com/parsing-html-the-cthulhu-way/) in most cases. HTML is a context-based language – the content described by any given HTML tag depends on where that tag is positioned relative to all other tags, and where that tag is opened and closed. Regex, on the other hand, ignores context. It goes along until it finds the conditions you've described and grabs the material there, ignoring everything else.  Combining HTML and regex creates [fragile code that breaks easily](http://stackoverflow.com/questions/6751105/why-its-not-possible-to-use-regex-to-parse-html-xml-a-formal-explanation-in-la).

Fortunately, there are context-aware parsers for HTML and XML that allow you to find what you need within the code without a headache.  Scrapy can work with any parser you provide, and natively incorporates two "selectors": [CSS-based and Xpath](https://doc.scrapy.org/en/latest/topics/selectors.html). We'll be using [Xpath](https://www.w3schools.com/xml/xpath_intro.asp) here: It treats nested code like computer file paths, with each level referred to as a 'node' (eg. C:\\User\Desktop\Stuff for files, \\body\header\p for code).

The Xpath expression to find the title of a submission in our page is: `//article/header/h2/a/@title`.  In English, this means:  

 * `//article`: _Locate every_ `article` _node in the page._
 * `/header`: _Find the_ `header` _node nested directly underneath the_ `article` _node._  
 * `/h2`: _Find the_ `h2` _node nested directly underneath the_ `header` _node_.  
 * `/a`: _Find the_ `a` _node nested directly underneath the_ `h2` _node._  
 * `/@title`: _Find the_ `title` _attribute within the_ `a` _node._  

The best way to learn Xpath is to try it out.  You can easily experiment using [this XPath tester](http://www.freeformatter.com/xpath-tester.html).

## Using JSON to store output

As we saw in the previous example, Scrapy writes information to a file.  But raw HTML isn't very useful. Here, we will use the [JSON file format](https://blog.datafiniti.co/4-reasons-you-should-use-json-instead-of-csv-2cac362f1943). JSON stores data in a format that looks a lot like Python dictionaries. This is nice for scraping where you are likely to get non-flat data (also called semi-structured data), where rows can have different numbers of columns. JSON files are also easy to read back into Python.

Because we're running Scrapy in a script, we need to call an instance of the CrawlerProcess class to actually run our scrapers. (In the API, this is taken care of automatically.) We can pass arguments into the class call that will give instructions to the scrapers, such as where to store the data (in this case, a JSON file called `firstpage.json`).

Remember, scrapy can't restart itself, so you have to restart the kernel before running this code.

In [3]:
# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = [
        'http://www.everydaysexism.com',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for article in response.xpath('//article'):
            
            # Yield a dictionary with the values we want.
            yield {
                # This is the code to choose what we want to extract
                # You can modify this with other Xpath expressions to extract other information from the site
                'name': article.xpath('header/h2/a/@title').extract_first(),
                'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
                'text': article.xpath('section[@class="entry-content"]/p/text()').extract(),
                'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }

# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'firstpage.json',  # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()
print('Success!')


Success!


# Checking the output

Did it work? We turned off logging in the example above, so the best way to check is to look at our output JSON file: `firstpage.json`.

**Note**: if you run the code more than once in a row, Scrapy will try to append the new scraped information to the end of the old JSON file. This will cause the JSON file to bug out because it will have an "end of file" notation in the middle of the file, so make sure to delete your JSON file before you re-run this or you'll get a "trailing file" error.

We can take a look at the raw json file (which is pretty human-readable) in your favorite text editor, or load it into Python as a data frame:

In [4]:
import pandas as pd

firstpage = pd.read_json('firstpage.json', orient='records')
print(firstpage.shape)
print(firstpage.head())

(10, 4)
        date      name                       tags  \
0 2017-08-23     Lucie                   [School]   
1 2017-08-23     Emily  [Public space, Workplace]   
2 2017-08-23  Bollocks                    [Media]   
3 2017-08-23  Samantha             [Public space]   
4 2017-08-23      Katy                         []   

                                                text  
0  [We didn’t have enough chairs in the classroom...  
1  [When I was 17, I worked in a clothing store t...  
2  [“Vaginas need sex or they’ll waste away” , Wi...  
3  [I was out at a bar with my boyfriend, my best...  
4  [I’m 17 and earlier this summer I was walking ...  


Success!  Ten rows (representing ten entries) each with date, name, tags, and text.  


## Recursion

Now that we have a scraper that can pull the information we want off of a page and store it in a file, we want to run that scraper over _all_ the pages of the website. We do this using recursion – the Scrapy spider will run over a page, gather information, and then detect a link to the next page and _call itself_ on the new page.  Then the new instance of the spider will do the same thing. This will repeat until it arrives at a page with no link to a next page, or when it reaches a stopping condition we define.

Since _Everyday Sexism_ has a paginated structure (every page links to the page before it), we can use recursion to get all the entries on the site. If you wanted to scrape a website with a non-linear structure, you'd still use the recursive setup, but with different restrictions on what links the scraper should follow. Scrapy is efficient and will only scrape each page once, even if it travels to them multiple times.

You may notice we've added some additional arguments to the ´CrawlerProcess()´ command- these have to do with scraping etiquette and will be introduced in the next assignment.

Again, you'll want to restart the kernel.

In [ ]:
# Importing in each cell because of the kernel restarts.
import scrapy
import re
from scrapy.crawler import CrawlerProcess

class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = [
        'http://www.everydaysexism.com',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for article in response.xpath('//article'):
            
            # Yield a dictionary with the values we want.
            yield {
                'name': article.xpath('header/h2/a/@title').extract_first(),
                'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
                'text': article.xpath('section[@class="entry-content"]/p/text()').extract(),
                'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }
        # Get the URL of the previous page.
        next_page = response.xpath('//div[@class="nav-previous"]/a/@href').extract_first()
        
        # There are a LOT of pages here.  For our example, we'll just scrape the first 9.
        # This finds the page number. The next segment of code prevents us from going beyond page 9.
        pagenum = int(re.findall(r'\d+',next_page)[0])
        
        # Recursively call the spider to run on the next page, if it exists.
        if next_page is not None and pagenum < 10:
            next_page = response.urljoin(next_page)
            # Request the next page and recursively parse it the same way we did above
            yield scrapy.Request(next_page, callback=self.parse)

# Tell the script how to run the crawler by passing in settings.
# The new settings have to do with scraping etiquette.          
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'data.json',       # Name our storage file.
    'LOG_ENABLED': False,          # Turn off logging for now.
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()
print('Success!')

In [ ]:
import pandas as pd

# Checking whether we got data from all 9 pages
ESSdf=pd.read_json('data.json', orient='records')
print(ESSdf.shape)
print(ESSdf.head())

# Conclusion

Nine pages at 10 entries a row gives us 90 rows - looks like we were successful in our scraping!  This is just a start. Scrapy is very flexible and will let you run multiple spiders with all sorts of different abilities.  In the next assignment, we'll talk about some of the options you can use to fine-tune scrapy and how to use your newfound scraping power in a responsible way.